### TOPOLOGICAL SORTING :

In [14]:
import networkx as nx
import pandas as pd
def combinational_ckt(filename):
    f=open(filename,'r')
    lis=[]
    for each in f:
        lis.append(each.split())
    f.close()
    return lis
def ip(filename):
    f=open(filename,'r')
    ip_nodes=[]
    inp=[]
    for each in f:
        if ip_nodes==[]:
            ip_nodes=list(each.split()).copy()
        else :
            inp.append(list(map(int,each.split())))
    return ip_nodes,inp
def unique(list):
    uni=[]
    for i in list:
        if i not in uni:
            uni.append(i)
    return uni
            

- combinational_ckt function is used to read the netlist and returning a list contains each line as a sublist and sublist contains the elements which are obtained ny splitting the line by spaces.
- ip function is used to read the input files and assigning input nodes and input values for different instants
- unique function is used to remove the duplicates in list and returns the list of distinct elements.

In [389]:
#creating a DAG
def toposorting(net_filename,input_filename):
    g=nx.DiGraph()
    #net=combinational_ckt("parity.net")
    net=combinational_ckt(net_filename)
    #print(net)
#EDGES
    edges=[]
    for i in range(len(net)):
        edges.append((net[i][2],net[i][-1]))
        if net[i][3]!=net[i][-1] :edges.append((net[i][3],net[i][-1]))
    #print(edges)
#NODES
    nodes=[]
    for i in range(len(net)):
        if net[i][2] not in nodes:
            nodes.append(net[i][2])
        if net[i][3] not in nodes:
            nodes.append(net[i][3])
        if net[i][-1] not in nodes:
            nodes.append(net[i][-1])
    #print(nodes)
#OUTPUTNODES
    output_nodes=[net[i][-1] for i in range(len(net))]
    #print(output_nodes)
#PRIMARY NODES
    primary_nodes=[i for i in nodes if i not in output_nodes]
#   print(primary_nodes)
#NODES NUMBERING
    nod_dict={}
    for i in range(len(net)):
        nod_dict[net[i][-1]]=net[i][1]
    for i in primary_nodes:
        nod_dict[i]='PI'
#   print(nod_dict)
#CONSTRUCTING GRAPH
    g.add_edges_from(edges)
    nx.set_node_attributes(g,nod_dict,name="gateType")
 #   print(g.nodes(data=True))
#TOPOLOGICAL SORT
    try :
        topo_nodes=list(nx.topological_sort(g))
    except : 
        topo_nodes=[]
        print('Graph contains a cycle or graph changed during iteration')
        return
#print(topo_nodes)
#ORDER OF TOPONODES
    topo_dict={}
    for i in range(len(topo_nodes)):
        topo_dict[topo_nodes[i]]=i
#print(topo_dict)
    topo_net=[]
    for i in range(len(topo_nodes)):
        for j in range(len(net)):
            if topo_nodes[i]==net[j][-1] : topo_net.append(net[j])
    ip_nodes,inp=ip(input_filename)
#print(topo_net)
    out=[]
    for i in inp:
        output_list=[0 for i in range(len(topo_dict))]
        pi_dict={}
        for j in range(len(ip_nodes)):
            pi_dict[ip_nodes[j]]=i[j]
        for key in pi_dict :
            output_list[topo_dict[key]]=pi_dict[key]
        for j in range(len(topo_net)):
            if topo_net[j][1]=='and2': 
                output_list[topo_dict[topo_net[j][-1]]]=output_list[topo_dict[topo_net[j][2]]] and output_list[topo_dict[topo_net[j][3]]]
            if topo_net[j][1]=='or2': 
                output_list[topo_dict[topo_net[j][-1]]]=output_list[topo_dict[topo_net[j][2]]] or output_list[topo_dict[topo_net[j][3]]]
            if topo_net[j][1]=='nand2': 
                output_list[topo_dict[topo_net[j][-1]]]=output_list[topo_dict[topo_net[j][2]]] and output_list[topo_dict[topo_net[j][3]]]
                output_list[topo_dict[topo_net[j][-1]]]=output_list[topo_dict[topo_net[j][-1]]]^1
            if topo_net[j][1]=='nor2': 
                output_list[topo_dict[topo_net[j][-1]]]=output_list[topo_dict[topo_net[j][2]]] or output_list[topo_dict[topo_net[j][3]]]
                output_list[topo_dict[topo_net[j][-1]]]=output_list[topo_dict[topo_net[j][-1]]]^1
            if topo_net[j][1]=='xor2': 
                output_list[topo_dict[topo_net[j][-1]]]=output_list[topo_dict[topo_net[j][2]]]^output_list[topo_dict[topo_net[j][3]]]
            if topo_net[j][1]=='xnor2': 
                output_list[topo_dict[topo_net[j][-1]]]=output_list[topo_dict[topo_net[j][2]]]^output_list[topo_dict[topo_net[j][3]]]
                output_list[topo_dict[topo_net[j][-1]]]=output_list[topo_dict[topo_net[j][-1]]]^1
            if topo_net[j][1]=='inv':
                output_list[topo_dict[topo_net[j][-1]]]=output_list[topo_dict[topo_net[j][2]]]^1
            if topo_net[j][1]=='buf':
                output_list[topo_dict[topo_net[j][-1]]]=output_list[topo_dict[topo_net[j][2]]]
        out.append(output_list)
    return out,topo_nodes



- Firstly we create a directional graph as in the presentation pdf.
- Edges are created by connecting the third element to the last and if net has one more input, forth element to the last element.
- nodes are the combination of input nodes (ususally third and forth element) and last element of each list.
- output nodes are the last element of each list.
- primary nodes are the nodes other than output nodes
- As mentioned in the pdf, I created the graph g and found top_nodes i.e., topological sort of the given nodes using topological_sort function
- Topo_dict is created numbering the topo_nodes such that element corresponding to the topo_list can easily be accessed.
- I have arranged the net in the order that output of elements are in topological order
- So that we can easily solve the circuit using a for loop on the topo_net and we can find every output of the gate and stored in the output_list.
- For every input of the inputs file, corresponding output_list appends to the out which is the final output list for the given input vectors.
- If there is any error in the inputs, error meaasge is printed.
- To avoid error when we are calling the function, I used excpetion handling.

In [390]:
try: out,topo_nodes=toposorting('c17.net','c17.inputs')
except : print("ERROR") # tot avoid error when unpacking the out and topo_nodes
if type(out)==list:
    out.insert(0,topo_nodes)
# displaying data
#for i in out:
#    p=' '.join(list(map(str,[x for x in i])))
#   print(p)
#SORTING THE COLUMNS OF OUT
out_dict={}
for i in range(len(out[0])):
    out_dict[out[0][i]]=[out[k][i] for k in range(1,len(inp)+1)]
#print(out_dict)
s_or=sorted(topo_nodes) #sorted order of topo_nodes
#print(s_or)
sorted_out={s:out_dict[s] for s in s_or}
#print(sorted_out)

- out_dict is constructed such that we can easily sort the columns correpeonding to 'out' using keys

In [391]:
topo_df=pd.DataFrame(sorted_out)
print(topo_df)

   N1  N2  N22  N23  N3  N6  N7  n_0  n_1  n_2  n_3
0   0   1    1    1   0   0   0    1    1    1    0
1   0   0    0    0   1   0   0    1    1    1    1
2   1   0    0    0   0   0   0    1    1    1    1
3   0   0    0    0   1   1   1    1    0    1    1
4   1   1    1    0   1   1   1    0    0    1    1


In [392]:
topo_df.to_excel('topo_output_file_c17.xlsx','w')

- I used pandas to create a dataframe feeding the sorted dictionary, such that the nodes are in alphabetic order.
- Created an excel file of the dataframe using to_excel function of pandas.
- I used writing mode such that if there is no file in the directory with the filename given, it creates a new file with data or else it relaces the data of the file with this data.
- You may get error if the file is open.

### EVENT DRIVEN EVALUATION :

In [393]:
def event_driven(net_filename,input_filename):
    g=nx.DiGraph()
    #net=combinational_ckt("parity.net")
    net=combinational_ckt(net_filename)
    #print(net)
#EDGES
    edges=[]
    for i in range(len(net)):
        edges.append((net[i][2],net[i][-1]))
        if net[i][3]!=net[i][-1] :edges.append((net[i][3],net[i][-1]))
    #print(edges)
#NODES
    nodes=[]
    for i in range(len(net)):
        if net[i][2] not in nodes:
            nodes.append(net[i][2])
        if net[i][-2] not in nodes:
            nodes.append(net[i][3])
        if net[i][-1] not in nodes:
            nodes.append(net[i][-1])
    #print(nodes)
#OUTPUTNODES
    output_nodes=[net[i][-1] for i in range(len(net))]
    #print(output_nodes)
#PRIMARY NODES
    primary_nodes=[i for i in nodes if i not in output_nodes]
    #print(primary_nodes)
    nod_dict={}
    for i in range(len(net)):
        nod_dict[net[i][-1]]=net[i][1]
    for i in primary_nodes:
        nod_dict[i]='PI'
#CONSTRUCTING GRAPH
    g.add_edges_from(edges)
    nx.set_node_attributes(g,nod_dict,name="gateType")
    if list(nx.simple_cycles(g))!=[]: #step*
        print("Graph contains a cycle or graph changed during iteration")
        return
    #print(g)
#INPUTS
    ip_nodes,inp=ip(input_filename)
    for i in output_nodes:
        for each in list(g.successors(i)):
            if each in ip_nodes:
                print("error")
                return
    q_dict={}
    for i in range(len(nodes)):
        q_dict[nodes[i]]=i
    #print(q_dict)
    #step1
    q_upd=[]
    update1=['x' for i in range(len(nodes))]
    q_upd.append(update1)
    op=[]
    for i in inp:
        flag=0
        qu=ip_nodes.copy() #step2
        pi_dict={} #dictionary of input nodes corresponding to its values of the input vector
        for j in range(len(ip_nodes)):
            pi_dict[ip_nodes[j]]=i[j]
        while qu!=[]:
            flag+=1
            if op==[]:
                if flag==1:
                    for j in range(len(i)):
                        suc=[]
                        u=q_upd[-1].copy()
                        suc=list(g.successors(qu[0]))
                        u[q_dict[qu[0]]]=pi_dict[qu[0]]
                        q_upd.append(u)#step 3
                        for each in suc:qu.append(each)#step4  
                        qu.pop(0) 
                if q_upd[-1][q_dict[qu[0]]]=='x':
                    suc=[]
                    u=q_upd[-1].copy()
                    suc=list(g.successors(qu[0]))
                    for each in suc:
                        qu.append(each)
                    gate=g.nodes[qu[0]]['gateType'] #gate type of qu[0]
                    pred=list(g.predecessors(qu[0]))#inputs to the gate type
                    if type(q_upd[-1][q_dict[pred[0]]])!=int or type(q_upd[-1][q_dict[pred[-1]]])!=int: #step 5
                        qu.pop(0)
                        continue
                    if gate=='and2': #step 6
                        u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]] and q_upd[-1][q_dict[pred[1]]]
                    if gate=='or2':
                        u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]] or q_upd[-1][q_dict[pred[1]]]
                    if gate=='xor2':
                        u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]]^q_upd[-1][q_dict[pred[1]]]
                    if gate=='nor2':
                        u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]] or q_upd[-1][q_dict[pred[1]]]
                    if gate=='nand2':
                        u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]] and q_upd[-1][q_dict[pred[1]]]
                        u[q_dict[qu[0]]]=u[q_dict[qu[0]]]^1
                    if gate=='xnor2':
                        u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]]^q_upd[-1][q_dict[pred[1]]]
                        u[q_dict[qu[0]]]=u[q_dict[qu[0]]]^1
                    if gate=="inv":
                        u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]]^1
                    if gate=="buf":
                        u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]]
                        #break
                       # k+=1
                    q_upd.append(u)   #step 7
               # q.pop(0)
            else:
                if flag==1 :
                    j=0
                    while j<len(i):  #step 8
                        suc=[]
                        u=q_upd[-1].copy()
                        u[q_dict[qu[0]]]=pi_dict[qu[0]]
                        q_upd.append(u)     
                        if q_upd[-1][q_dict[qu[0]]]!=q_upd[-2][q_dict[qu[0]]] : 
                            suc=list(g.successors(qu[0]))
                        for each in suc:qu.append(each)
                        qu.pop(0)
                        j+=1
                qu=unique(qu) #step 9
                u=q_upd[-1].copy()
                gate=g.nodes[qu[0]]['gateType'] #gate type of qu[0]
                pred=list(g.predecessors(qu[0])) #inputs to the gate type
                if gate=='and2': #step6
                    u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]] and q_upd[-1][q_dict[pred[1]]]
                if gate=='or2':
                    u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]] or q_upd[-1][q_dict[pred[1]]]
                if gate=='xor2':
                    u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]]^q_upd[-1][q_dict[pred[1]]]
                if gate=='nor2':
                    u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]] or q_upd[-1][q_dict[pred[1]]]
                if gate=='nand2':
                    u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]] and q_upd[-1][q_dict[pred[1]]]
                    u[q_dict[qu[0]]]=u[q_dict[qu[0]]]^1
                if gate=='xnor2':
                    u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]]^q_upd[-1][q_dict[pred[1]]]
                    u[q_dict[qu[0]]]=u[q_dict[qu[0]]]^1
                if gate=="inv":
                    u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]]^1
                if gate=="buf":
                    u[q_dict[qu[0]]]=q_upd[-1][q_dict[pred[0]]]
                q_upd.append(u)
                suc=[]
                if q_upd[-1][q_dict[qu[0]]]!=q_upd[-2][q_dict[qu[0]]] : suc=list(g.successors(qu[0]))#step 10
                for each in suc: qu.append(each)
            qu.pop(0)
        op.append(q_upd[-1])  #step o        
    return op,nodes



- Constructing the graph is same as presentations.
- q_upd is the state table of nodes.
- op is the final output list of all input vectors.
- q_dict is used for the numbering of the nodes.
- qu is the queue of nodes to be executed.
- u is the update need to be appended to the q_upd when required.
- In event driven evaluation, for the first input vector we need to find the outputs of every node.
- From the second input vector, if there is change in the value of the primary inputs, we need to find the successors of the node corresponding to that primary input until the last successor or it gets stopped at any node where there is no change in it's value with respect to it's previous value.
- step*: nx.simple_cycles(g) find out the cycles in graph. if there is a cycle it is not an empty list so, it throws an error and returns. if there is no cycle it proceeds.
- step1:firstly we will make every value of the node in the update1 to be'x' and append it to the q_upd.
- step 2: for the first input vector, add all the elements of the input vector to the qu and create pi_dict whish is the dictionary of primary input nodes corresponding to its values of the input vector.
- step 3: update the every change in the primary input node values one by one to the q_upd
- step 4: add all the successors of the primary input nodes to the qu and pop out the primary input nodes correspondingly.
- step 5:if the inputs qu[0] are not integers, pop it out and continue with the remaining elements in qu.
- step 6: find the gate type of qu[0] and inputs and execute the q[0] gate.Append it's successors too and step 5 will be repeated.
- step 7: update the q_upd and pop the first element.it will get repeated until qu becones empty.After that qu.is updated with new set of inputs.
- step o :after the first input vector is done, now the last element of q_upd no more conatins 'x' and it contains the output of the first element. So, it is appened to op
- step 8: now we will check every primary input with the value corresponding to the q_up[-1]. If there is a change it's values are updated and it's successors will be added to the and append the update and pop the first element.
- step 9: As every input is updated in q_upd, we can take distinct elements of qu and execute them. This is done using unique() function and repeat step 5
- step 10: after calculating the qu[0] , update the q_upd, If it's value in q_upd[-1] and q_upd[-2] are not equal it's successors are added and pop the first element of qu

In [412]:

try:
    op,nodes=event_driven('parity.net','parity.inputs')   
except :print("error")

op.insert(0,nodes)
o_dict={}
for i in range(len(op[0])):
    o_dict[op[0][i]]=[op[k][i] for k in range(1,len(inp)+1)]
#print(o_dict)
s_or=sorted(nodes)
#print(s_or)
sorted_out_event={s:o_dict[s] for s in s_or}
#print(sorted_out_event)


In [413]:
event_df=pd.DataFrame(sorted_out_event)
print(event_df)

   a  b  c  d  dummy_0  dummy_1  dummy_2  dummy_3  dummy_4  e  ...  n_2  n_3  \
0  0  1  0  0        0        0        0        0        0  0  ...    0    0   
1  0  0  1  0        0        1        0        0        1  0  ...    1    1   
2  1  0  0  0        0        0        0        1        0  0  ...    0    1   
3  0  0  1  1        1        1        0        1        0  1  ...    1    1   
4  1  1  1  1        1        0        1        1        0  1  ...    0    1   

   n_4  n_5  n_6  n_7  n_8  o  p  q  
0    0    1    1    1    0  1  1  1  
1    0    1    0    0    1  1  1  0  
2    1    1    1    1    0  0  0  1  
3    0    0    1    0    0  1  1  0  
4    1    0    1    0    1  0  1  1  

[5 rows x 31 columns]


In [414]:
event_df.to_excel('event_driven_output_file_parity.xlsx','w')

In [397]:
%timeit toposorting('c17.net','c17.inputs')
%timeit event_driven('c17.net','c17.inputs')

302 µs ± 13.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
488 µs ± 16.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [398]:
%timeit toposorting('c8.net','c8.inputs')
%timeit event_driven('c8.net','c8.inputs')

2.71 ms ± 58.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.64 ms ± 128 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [399]:
%timeit toposorting('c432.net','c432.inputs')
%timeit event_driven('c432.net','c432.inputs')

24.2 ms ± 3.13 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.81 s ± 207 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [400]:
%timeit toposorting('parity.net','parity.inputs')
%timeit event_driven('parity.net','parity.inputs')

796 µs ± 51.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.44 ms ± 182 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


- As observed, topo_sorting is nore efficient than event_driven because time taken by event_driven is very large compared to topo_sorting.
- As the no. of gates in a netlist increasing, the time taken by both the evaluations increasing.
- Event_driven evaluation is completely dependent no. of gates involved with the changes in the input

###### NOTE:
- First import the libraries which u require.
- Run the user defined function before you use it in some other cell.

- This is an .ipynb file, which we can run in Jupyter notebook or Jupyter lab.
- In Jupyter lab, we need to upload this document to the workspace and start editting and running.
- In jupyter notebook, which is a local host of our pc, we need to know where the file is located, opening this file is same as we do in file manager.
- To open in jupyter notebook, first we need to unzip the file i.e., extract all the files from it and open the ipynb file.